#[Your_name]

<h1 style="text-align: center;">Text Classification<h1>

Prepare libraries

In [253]:
# here put every import you need e.g. import nltk
# it's better to load what you need from the package by from [] import [] instead of import the whole package
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Conv1D, AveragePooling1D, Flatten
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
import re
import numpy as np
from cleantext import clean
from arabicstopwords.stopwords_lexicon import stopwords_lexicon
import arabicstopwords.arabicstopwords as stop_words
import nltk
from wordcloud import WordCloud
from arabic_reshaper import arabic_reshaper
from bidi.algorithm import get_display
from nltk import bigrams, trigrams
from collections import Counter
from snowballstemmer import stemmer
from tashaphyne.stemming import ArabicLightStemmer
from farasa.segmenter import FarasaSegmenter
from farasa.stemmer import FarasaStemmer
from matplotlib.font_manager import FontProperties
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score

from nltk.tokenize import word_tokenize


download 

Download data

In [254]:
# !wget 'https://drive.google.com/uc?export=download&id=1cMSjxa3nA706LIZDEhwMpaVRMY2IX9P0' -O 'data.zip'

In [255]:
# !pip install Arabic-Stopwords
# 2.2 nltk.download('punkt') # open vpn
# 2.3 !pip install arabic-reshaper
# 2.4 nltk.download('stopwords')
# 3.8 !pip install snowballstemmer
# 3.8 !pip install Tashaphyne
# 3.8 !pip install -U farasapy

In [256]:
# !unzip data.zip

<h2 dir="rtl">مثال عن كيفية تنظيف حلول الطلبات:</h2>


<div dir="rtl">شرح ما يقوم به الكود (like code documentation)<div>

In [257]:
# your code here


In [258]:
# example test

<div dir="rtl">ملاحظاتك في حال وجودها</div>
<div dir="rtl">يمكنك إضافة خلايا لكل طلب بقدر ما تشاء، المهم أن تحافظ على تنظيم الملف</div>

# Question [1]: Load the data

## [1.1]

In [259]:
train_file_path = 'train.csv'
test_file_path = 'test.csv'
validation_file_path = 'val.csv'
train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)
validation_data = pd.read_csv(validation_file_path)

# print("number of data in train data : ",len(train_data))
# print("number of data in test data : ",len(test_data))
# print("number of data in validation data : ",len(validation_data))

In [260]:
# train_data.head(10)

In [261]:
# test_data.head(10)

In [262]:
# validation_data.head(10)

In [263]:
#drop_duplicates يزيل الصفوف المتكرره
#dropna يحذف الصفوف المكررة
test_data=test_data.drop_duplicates().dropna()
train_data=train_data.drop_duplicates().dropna()
validation_data=validation_data.drop_duplicates().dropna()
# print("number of data in train data after cleaning : ",len(train_data))
# print("number of data in test data after cleaning : ",len(test_data))
# print("number of data in validation data after cleaning : ",len(validation_data))

In [264]:
# train_data.head(10)

In [265]:
# test_data.head(10)

In [266]:
# validation_data.head(10)

## [1.2]

In [267]:
def unify_specialty(name):
    name = re.sub(r"[^\w\s]", " ", name)  # يزيل الرموز الترقيمية
    name = re.sub(r"\s+", " ", name).strip()  # يزيل الفراغات المتكرره #strip يزيل الفراغات يلي بالبداية والنهاية
    name =re.sub("[\_\-]"," ",name)
    return name

train_data['label']=train_data['label'].apply(unify_specialty)
test_data['label']=test_data['label'].apply(unify_specialty)
validation_data['label']=validation_data['label'].apply(unify_specialty)   


In [268]:
# x=set(train_data['label'])
# x

In [269]:
Dictionary_of_diseases= {
    'أمراض الجهاز التنفسي': 'أمراض الجهاز التنفسي',
    'الجهاز التنفسي': 'أمراض الجهاز التنفسي',
    'امراض الجهاز التنفسي': 'أمراض الجهاز التنفسي',
    'أمراض الدم': 'أمراض الدم',
    'الدم': 'أمراض الدم',
    'امراض الدم': 'أمراض الدم',
    'أمراض الغدد الصماء': 'أمراض الغدد الصماء',
    'الغدد الصماء': 'أمراض الغدد الصماء',
    'امراض الغدد الصماء': 'أمراض الغدد الصماء',
    'مرض السكري': 'مرض السكري',
    'السكري': 'مرض السكري',
    'الاورام الخبيثة والحميدة':'الأورام الخبيثة والحميدة'

}

def Unification_name_of_diseases(c):
    c=c.map(Dictionary_of_diseases).fillna(c)
    return c

In [270]:
train_data['label']=Unification_name_of_diseases(train_data['label'])
test_data['label']=Unification_name_of_diseases(test_data['label'])
validation_data['label']=Unification_name_of_diseases(validation_data['label'])

In [271]:
# x=set(train_data['label'])
# x

# Question [2]: Text Analysis

## [2.1] 


In [272]:
# train_data

In [273]:
# Train data

# عدد الأجوبة الخاصة بأمراض الدم :
answers_len_in_blood_diseases = len(train_data[train_data['label']=='أمراض الدم']['answer'].tolist())
# print(answers_len_in_blood_diseases)

# عدد الأسئلة الخاصة بأمراض الدم :
questions_len_in_blood_diseases = len(train_data[train_data['label']=='أمراض الدم']['question'].tolist())
# print(questions_len_in_blood_diseases)


# عدد الداتا الخاصة بأمراض الدم :
total_blood_diseases_len = len(train_data[train_data['label']=='أمراض الدم'])
print('عدد الداتا الخاصة بأمراض الدم:',total_blood_diseases_len)

# عدد الداتا في أمراض الجهاز التنفسي :
total_blood_diseases_len = len(train_data[train_data['label']=='أمراض الجهاز التنفسي'])
print('عدد الداتا الخاصة بأمراض الجهاز التنفسي:',total_blood_diseases_len)

# غدد الداتا في أمراض الغدد الصماء :
total_blood_diseases_len = len(train_data[train_data['label']=='أمراض الغدد الصماء'])
print('عدد الداتا الخاصة بأمراض الغدد الصماء:',total_blood_diseases_len)

# عدد  الداتا ارتفاع ضغط الدم :
total_blood_diseases_len = len(train_data[train_data['label']=='ارتفاع ضغط الدم'])
print('عدد الداتا الخاصة بأمراض ارتفاع ضغط الدم:',total_blood_diseases_len)

# عدد الداتا في الأورام الخبيثة والحميدة :
total_blood_diseases_len = len(train_data[train_data['label']=='الأورام الخبيثة والحميدة'])
print('عدد الداتا الخاصة بالأورام الخبيثة والحميدة:',total_blood_diseases_len)

# عدد الداتا في جراحة العظام
total_blood_diseases_len = len(train_data[train_data['label']=='جراحة العظام'])
print('عدد الداتا الخاصة بجراحةالعظام:',total_blood_diseases_len)

# عدد الداتا في الجراحة العامة
total_blood_diseases_len = len(train_data[train_data['label']=='جراحة عامة'])
print('عدد الداتا الخاصة بالجراحة العامة:',total_blood_diseases_len)

# عدد الداتا في مرض السكري
total_blood_diseases_len = len(train_data[train_data['label']=='مرض السكري'])
print('عدد الداتا الخاصة بمرض السكري:',total_blood_diseases_len)


عدد الداتا الخاصة بأمراض الدم: 1398
عدد الداتا الخاصة بأمراض الجهاز التنفسي: 3676
عدد الداتا الخاصة بأمراض الغدد الصماء: 5752
عدد الداتا الخاصة بأمراض ارتفاع ضغط الدم: 2537
عدد الداتا الخاصة بالأورام الخبيثة والحميدة: 6449
عدد الداتا الخاصة بجراحةالعظام: 2211
عدد الداتا الخاصة بالجراحة العامة: 6307
عدد الداتا الخاصة بمرض السكري: 4268


In [274]:
# # Drawing pie chart:
# # ملاحظة بالعربي طلع الخط مفشكل ف تمت الكتابة بالانكليزي



# data = {
#     'الاختصاص': ['أمراض الدم', 'أمراض الجهاز التنفسي', 'أمراض الغدد الصماء','أمراض ارتفاع ضغط الدم','الأمراض الخبيثة والحميدة','جراحةالعظام','الجراحة العامة','مرض السكري'],
#     'عدد النصوص': [2153, 6002, 9417,4190,10711,2641,10548,7096]
# }

# data


# df = pd.DataFrame(data)

# # print(df)
# arabic_font = FontProperties(fname='font/NotoNaskhArabic-VariableFont_wght.ttf') 

# df['الاختصاص'] = [get_display(arabic_reshaper.reshape(label)) for label in df['الاختصاص']]
# plt.figure(figsize=(8, 8))
# plt.pie(df['عدد النصوص'], labels=df['الاختصاص'], autopct='%1.1f%%', startangle=140)
# plt.title(get_display(arabic_reshaper.reshape('توزيع النصوص حسب الاختصاص')), fontproperties=arabic_font)
# plt.show()

# # import pandas as pd
# # import matplotlib.pyplot as plt

# # data = {
# #     'Specialization': ['Blood Diseases', 'Respiratory System Diseases', 'Endocrine Diseases', 'Hypertension', 'Malignant and Benign Diseases', 'Orthopedic Surgery', 'General Surgery', 'Diabetes'],
# #     'Number of Texts': [1398, 3676, 5752, 2537, 6449, 2211, 6307, 4268]
# # }

# # df = pd.DataFrame(data)

# # print(df)

# # plt.figure(figsize=(8, 8))
# # plt.pie(df['Number of Texts'], labels=df['Specialization'], autopct='%1.1f%%', startangle=140)
# # plt.title('Distribution of Texts by Specialization')
# # plt.show()


## [2.2] 

In [275]:
def delete_repeated_letter(text):
    # cleaned_text = re.sub(r'(.)\1{2,}', r'\1', text)
    cleaned_text = re.sub(r'([^\w\s\.])\1+', r'\1', text)
    cleaned_text = re.sub(r'(\.\s*)\1+$', r'\1', cleaned_text)
    return cleaned_text

def handle_connected_words(tokenized):
    # Handle connected words with '-'
    result_tokens = []
    for token in tokenized:
        # Split connected words with '-'
        result_tokens.extend(token.split('-'))
    return result_tokens


def filter_text(list):
    list_of_phrases =[]
    all_tokens = []
    for sentence in list:
        tokenizer = RegexpTokenizer(r'\b\d+\b|\b[^\d\W_]{2,}\b|[^\d\W_]+(?:-[^\d\W_]+)?|\S')
        tokenized = tokenizer.tokenize(delete_repeated_letter(sentence))
        tokenized = handle_connected_words(tokenized)

        # Remove '-' and '/' and one-letter words from each token
        # cleaned_tokens = [token for token in tokenized if len(token) > 1 and token not in ['-', '/']]
        # cleaned_tokens = [token if len(token) > 1 or token == 'و' else '' for token in tokenized if token not in ['-', '/']]

        cleaned_tokens = [token if (token.isalpha() and len(token) > 1) or token == 'و' else '' for token in tokenized if token not in ['-', '/']]

        combined_phrase = ' '.join(cleaned_tokens)
        list_of_phrases.append(combined_phrase)

    for phrase in list_of_phrases:
        tokens = nltk.word_tokenize(phrase)
        all_tokens.append(tokens)
        
    return all_tokens



In [276]:
train_data_answers_list = train_data['answer'].values
answers = filter_text(train_data_answers_list)
flattened_answers = [item for sublist in answers for item in sublist]
print(len(flattened_answers))


1645245


In [277]:
train_data_questions_list = train_data['question'].values
questions = filter_text(train_data_questions_list)
flattened_quesions = [item for sublist in questions for item in sublist]
print(len(flattened_quesions))

801979


In [278]:
# حساب عدد الكلمات الكلي :
total_words_len = len(flattened_quesions) + len(flattened_answers)
print(total_words_len)

2447224


In [279]:
# حساب عدد الكلمات الفريدة :
total_unique_words=len(set(flattened_quesions+flattened_answers))
print("عدد الكلمات الفريدة:", total_unique_words)

عدد الكلمات الفريدة: 94614


## [2.3] 

In [280]:
# WCD=dict()
# train_data_answers_list = flattened_answers
# train_data_questions_list = flattened_quesions
# train_data_list=train_data_answers_list+train_data_questions_list
# answers_text = ' '.join(train_data_list)
# reshaped_text = arabic_reshaper.reshape(answers_text)
# arabic_text = get_display(reshaped_text)
# wordcloud = WordCloud(font_path='font/NotoNaskhArabic-VariableFont_wght.ttf', background_color='white').generate(arabic_text)
# WCD=wordcloud.words_
# plt.figure(figsize=(50, 50))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.show()

## [2.4] 

In [281]:
def tokenized_text(text):
    # allWords = nltk.tokenize.word_tokenize(text)

    allWordDist = nltk.FreqDist(w for w in text)
    stopwords = nltk.corpus.stopwords.words('english')
    allWordExceptStopDist = nltk.FreqDist(w.lower() for w in text if w not in stopwords)  
    mostCommon= allWordDist.most_common(15)
    print('أكثر 15 كلمة مكررة:',mostCommon)
    leastCommon = allWordDist.most_common()[:-11:-1]
    print('أقل 10 كلمة مكررة:', leastCommon)
    bigram_freq = nltk.FreqDist(bigrams(text))
    most_common_bigrams = bigram_freq.most_common(10)
    print('أكثر 10 ثنائيات مكررة في النص:',most_common_bigrams)
    trigram_freq = nltk.FreqDist(trigrams(text))
    most_common_trigrams = trigram_freq.most_common(10)
    print('أكثر 10 ثلاثيات كلمات مكررة في النص:',most_common_trigrams)
    collocations_bigram = nltk.collocations.BigramCollocationFinder.from_words(text)
    most_common_collocations_bigram = collocations_bigram.nbest(nltk.collocations.BigramAssocMeasures.likelihood_ratio, 10)
    print('أكثر 10 ثنائيات كلمات مهمة في النص:',most_common_collocations_bigram)
    collocations_trigram = nltk.collocations.TrigramCollocationFinder.from_words(text)
    most_common_collocations_trigram = collocations_trigram.nbest(nltk.collocations.TrigramAssocMeasures.likelihood_ratio, 10)
    print('أكثر 10 ثلاثيات كلمات مهمة في النص:',most_common_collocations_trigram)  
    print('========================================================================================================================================================================================')  


    


# tokenized_text('tony botros is tony the tony best best best tony botros in in in the whole world tony botros')
# tokenized_text(['طوني','طوني','بطرس','هو','طوني','هو','بطرس','ال','الأفضل'])
# tokenized_text(['لدي', 'جرح', 'فوق', 'حاجبي', 'و', 'تمت', 'عملية', 'الخياطة', 'ثم', 'نزع', 'الغرزات', 'بشكل', 'عادي', 'لكن', 'مر', 'اسبوع', 'على', 'ذلك', 'و', 'مازال', 'هناك', 'الم', 'و', 'تطلب', 'شديد', 'في', 'المنطقة', 'لدرجة', 'اني', 'لا', 'استطيع', 'رفع', 'حاجبي', 'ابدا', 'إضافة', 'لوجود', 'انتفاخ', 'هل', 'هدا', 'طبيعي', 'ام', 'يمكن', 'ان', 'يكون', 'خلل', 'و', 'إن', 'كان', 'كذلك', 'ما', 'هو'])
tokenized_text(answers[0])

أكثر 15 كلمة مكررة: [('المريض', 2), ('قد', 2), ('يناسب', 2), ('الدكتور', 2), ('يزن', 2), ('علي', 2), ('خليف', 2), ('و', 2), ('لكل', 1), ('علاج', 1), ('ايجابيته', 1), ('وسلبياته', 1), ('والتي', 1), ('تعتمد', 1), ('على', 1)]
أقل 10 كلمة مكررة: [('الصماء', 1), ('الغدد', 1), ('الانسولين', 1), ('ايضا', 1), ('جلوكوفانس', 1), ('الدواء', 1), ('عيوب', 1), ('مميزات', 1), ('هي', 1), ('ما', 1)]
أكثر 10 ثنائيات مكررة في النص: [(('الدكتور', 'يزن'), 2), (('يزن', 'علي'), 2), (('علي', 'خليف'), 2), (('لكل', 'علاج'), 1), (('علاج', 'ايجابيته'), 1), (('ايجابيته', 'وسلبياته'), 1), (('وسلبياته', 'والتي'), 1), (('والتي', 'تعتمد'), 1), (('تعتمد', 'على'), 1), (('على', 'حالة'), 1)]
أكثر 10 ثلاثيات كلمات مكررة في النص: [(('الدكتور', 'يزن', 'علي'), 2), (('يزن', 'علي', 'خليف'), 2), (('لكل', 'علاج', 'ايجابيته'), 1), (('علاج', 'ايجابيته', 'وسلبياته'), 1), (('ايجابيته', 'وسلبياته', 'والتي'), 1), (('وسلبياته', 'والتي', 'تعتمد'), 1), (('والتي', 'تعتمد', 'على'), 1), (('تعتمد', 'على', 'حالة'), 1), (('على', 'حالة', 'المريض

In [282]:
#الشرح

In [283]:
# هاد التابع لمرق كل اختصاص وجيب منو الكلمات كلا متل مو مطلوب ب 2.4
def all_tokinized_text(specialized):
    filtered_questions = filter_text(train_data[train_data['label']==specialized]['question'].values)
    filtered_answers = filter_text(train_data[train_data['label']==specialized]['answer'].values)
    list = filtered_questions + filtered_answers
    flattened_list = [item for sublist in list for item in sublist]
    # flattened_list
    tokenized_text(flattened_list)
    

In [284]:
# all_tokinized_text('أمراض الدم')

In [285]:
# all_tokinized_text('أمراض الجهاز التنفسي')

In [286]:
# all_tokinized_text('أمراض الغدد الصماء')

In [287]:
# all_tokinized_text('ارتفاع ضغط الدم')

In [288]:
# all_tokinized_text('الأورام الخبيثة والحميدة')

In [289]:
# all_tokinized_text('جراحة العظام')

In [290]:
# all_tokinized_text('جراحة عامة')

In [291]:
# all_tokinized_text('مرض السكري')

In [292]:
# على كل الداتا تبع الترين:
filtered_questions = filter_text(train_data['question'].values)
filtered_answers = filter_text(train_data['answer'].values)
list = filtered_questions + filtered_answers
flattened_list = [item for sublist in list for item in sublist]
flattened_list
tokenized_text(flattened_list)


أكثر 15 كلمة مكررة: [('من', 53731), ('في', 47548), ('الدكتور', 44651), ('طبية', 32856), ('اسئلة', 32618), ('و', 27315), ('جراحة', 20023), ('هل', 19845), ('الدم', 18946), ('على', 17807), ('امراض', 14809), ('عامة', 14630), ('لا', 13177), ('الغدد', 12601), ('مع', 12048)]
أقل 10 كلمة مكررة: [('قسوى', 1), ('إجبار', 1), ('إتباعه', 1), ('ومسكنة', 1), ('والمتعبة', 1), ('للنتوءات', 1), ('خياراتها', 1), ('Spurs', 1), ('بافراازات', 1), ('لوتتسبب', 1)]
أكثر 10 ثنائيات مكررة في النص: [(('اسئلة', 'طبية'), 32598), (('جراحة', 'عامة'), 14448), (('الغدد', 'الصماء'), 11130), (('طبية', 'امراض'), 10883), (('طبية', 'جراحة'), 8607), (('ضغط', 'الدم'), 6735), (('اعاني', 'من'), 6247), (('الاورام', 'الخبيثة'), 6106), (('الخبيثة', 'والحميدة'), 6091), (('طبية', 'الاورام'), 6089)]
أكثر 10 ثلاثيات كلمات مكررة في النص: [(('اسئلة', 'طبية', 'امراض'), 10883), (('اسئلة', 'طبية', 'جراحة'), 8607), (('اسئلة', 'طبية', 'الاورام'), 6089), (('طبية', 'الاورام', 'الخبيثة'), 6089), (('الاورام', 'الخبيثة', 'والحميدة'), 6089), (('طب

In [293]:
#الشرح

## [2.5] 

In [294]:
train_data['all_word']=train_data['answer']+train_data['question']
train_data_list=train_data['all_word'].values
all_word = filter_text(train_data_list)

In [295]:
flattened_all_word = [item for sublist in all_word for item in sublist]
train_data['length_of_sentinse']=train_data['all_word'].apply(len)
# train_data['length_of_sentinse']

In [296]:
w=[]
for i in all_word:
    w.append(set(i))
train_data['unique_words']=w
train_data['number_of_unique_words']=train_data['unique_words'].apply(len)

In [297]:
def histogram(length_of_sentinse,number_of_unique_words):
    plt.figure(figsize=(10, 6))
    plt.subplot(2, 1, 1)
    # plt.bar(number_of_unique_words, length_of_sentinse, color='blue', alpha=0.7)
    plt.hist(length_of_sentinse, label='length_of_sentinse',edgecolor='black', linewidth=1.2,bins=30, alpha=0.5)
    plt.hist(number_of_unique_words, label='number_of_unique_words',edgecolor='black', linewidth=1.2,bins=30, alpha=0.5)
    plt.title('histogram')
    plt.xlabel('length_of_sentinse')
    plt.ylabel('number_of_unique_words')
    # plt.tight_layout()
    plt.legend()
    plt.show()

In [298]:
# لكل النص
# histogram(train_data['length_of_sentinse'],train_data['number_of_unique_words'])

In [299]:
# # أمراض الجهاز التنفسي
# histogram(train_data[train_data['label']=='أمراض الجهاز التنفسي']['length_of_sentinse'],train_data[train_data['label']=='أمراض الجهاز التنفسي']['number_of_unique_words'])

In [300]:
# # أمراض الدم
# histogram(train_data[train_data['label']=='أمراض الدم']['length_of_sentinse'],train_data[train_data['label']=='أمراض الدم']['number_of_unique_words'])

In [301]:
# # أمراض الغدد الصماء
# histogram(train_data[train_data['label']=='أمراض الغدد الصماء']['length_of_sentinse'],train_data[train_data['label']=='أمراض الغدد الصماء']['number_of_unique_words'])

In [302]:
# # ارتفاع ضغط الدم
# histogram(train_data[train_data['label']=='ارتفاع ضغط الدم']['length_of_sentinse'],train_data[train_data['label']=='ارتفاع ضغط الدم']['number_of_unique_words'])

In [303]:
# # الأورام الخبيثة والحميدة
# histogram(train_data[train_data['label']=='الأورام الخبيثة والحميدة']['length_of_sentinse'],train_data[train_data['label']=='الأورام الخبيثة والحميدة']['number_of_unique_words'])

In [304]:
# # جراحة العظام
# histogram(train_data[train_data['label']=='جراحة العظام']['length_of_sentinse'],train_data[train_data['label']=='جراحة العظام']['number_of_unique_words'])

In [305]:
# # جراحة عامة
# histogram(train_data[train_data['label']=='جراحة عامة']['length_of_sentinse'],train_data[train_data['label']=='جراحة عامة']['number_of_unique_words'])

In [306]:
# # مرض السكري
# histogram(train_data[train_data['label']=='مرض السكري']['length_of_sentinse'],train_data[train_data['label']=='مرض السكري']['number_of_unique_words'])

# Question [3]: Text Cleaning and Normalization

## [3.1] 


In [307]:
# train_data
# clean_tweet = re.sub(r'^RT(\s)+|https?\S+|#|@\S+', '', tweet) ## من أجل أزالت 
#r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def remove_links(text):
    clean_text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    return clean_text


In [308]:
# remove_links('https://chat.openai.com/c/dc21a08a-5be6-42e3-a6e4-0c1d26791a46 hi')

## [3.2] 

In [309]:
import re

def remove_doctors_prefix(text):
    pattern = r'\b(?:الدكتور(?:ة)?)\s+(?:الصيدلاني(?:ة)?)?\s*(?:([\u0600-\u06FF]+)\s*[\u0600-\u06FF]+\s*([\u0600-\u06FF]+))\b'

    def replacer(match):
        full_name = match.group(1)
        first_name = full_name.split()[0]
        return f'د.{first_name}'

    result = re.sub(pattern, replacer, text)

    return result


In [310]:
texts = train_data[train_data['label']=='أمراض الدم']['answer']
# texts.apply(remove_doctors_prefix)[0]
# for text in texts[:5]:
#     # print(text)
#     print(remove_doctors_prefix(text))
#     print("=================================================================")

## [3.3] 

In [311]:
def remove_sentences_after_doctor(text):
    result = re.sub(r'(الدكتور[^\n]*\n|د\.[^\n]*\n)([^\n]*\n){0,6}', r'\1', text)
    return result

# for text in texts[:5]:
#     print(remove_sentences_after_doctor(text))

## [3.4] 

In [312]:
def remove_duplicate_letters_except_alif(text):
    pattern = re.compile(r'(?:(?<!ا)ا(?!ا)|(.)(?:\1)+)', re.UNICODE) # بشلي كلشي أحرف مكررة وخاصة الألف مابكررها اكتر من مرة
    result = pattern.sub(lambda x: x.group(1) if x.group(1) else 'ا', text)
    return result

# for text in texts[:5]:
#     print(remove_duplicate_letters_except_alif(text))

## [3.5]

### [3.5.1] 

In [313]:
# هاد بيقلب لل 1و2و3و4و....
def convert_numbers_to_arabic_v2(text):
    digit_mapping = {'٠': '0', '١': '1', '٢': '2', '٣': '3', '٤': '4', '٥': '5', '٦': '6', '٧': '7', '٨': '8', '٩': '9'}
    english_text = re.sub(r'[٠-٩]', lambda x: digit_mapping[x.group()], text)

    return english_text


# for text in texts[:5]:
#     print(convert_numbers_to_arabic_v2(text))

In [314]:
# هاد بيلقلب لل ١,٢,٣,٤
def convert_numbers_to_arabic(text):
    digit_mapping = {'0': '٠', '1': '١', '2': '٢', '3': '٣', '4': '٤', '5': '٥', '6': '٦', '7': '٧', '8': '٨', '9': '٩'}
    arabic_text = re.sub(r'[0-9]', lambda x: digit_mapping[x.group()], text)

    return arabic_text


# for text in texts[:5]:
#     print(convert_numbers_to_arabic(text))

### [3.5.2] 

In [315]:
def standardize_numbers(text, replacement_symbol='T'):
    digit_pattern = r'[0-9٠-٩]+'
    standardized_text = re.sub(digit_pattern, lambda x: replacement_symbol, text)

    return standardized_text


# for text in texts[:5]:
#     print(standardize_numbers(text))

### [3.5.3] 

In [316]:
def remove_numbers(text):
    digit_pattern = r'\d+'
    
    text_without_numbers = re.sub(digit_pattern, '', text)

    return text_without_numbers

# for text in texts[:5]:
#     print(remove_numbers(text))

### [3.5.4]

In [317]:
texts = train_data[train_data['label']=='أمراض الجهاز التنفسي']['answer'].values

def remove_dates_and_times(text):
    date_time_patterns = [
        r'\d{4}-\d{2}-\d{2}\s+\d{1,2}:\d{2}:\d{2}'  # Match dates and times like 2015-01-04 20:08:51
        # r'\d{1,2}/\d{1,2}/\d{2,4}',         # Match dates like 12/31/2022
        # r'\d{1,2}-\d{1,2}-\d{2,4}',         # Match dates like 12-31-2022
        # r'\d{1,2}:\d{2}:\d{2}'               # Match times like 12:34:56
        # r'\d{1,2}\s+(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s*\d{2,4}',  # Match dates like 31 Dec 2022
        # r'\d{1,2}:\d{2}\s*(?:AM|PM|am|pm)?',  # Match times like 12:34 AM
    ]

    for pattern in date_time_patterns:
        text = re.sub(pattern, '', text)

    return text

# for text in texts[:5]:
#     print(remove_dates_and_times(text))

## [3.6] 

### [3.6.1] 

In [318]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [319]:
def remove_punctuation(text):
    text=re.sub(r'[!\"#$%&\'()*+,\-./:;<=>?@\[\\\]^_`{|}~،؛؟ـ]',' ',text)
    return text

# for text in texts[:5]:
#     print(remove_punctuation(text))

### [3.6.2] 

In [320]:
def keep_arabic_punctuations_numbers(text):
    text= re.sub(r'[^\u0600-\u06FF0-9،؛؟ـ!\"#$%&\'()*+,\-./:;<=>?@\[\\\]^_`{|}~]', ' ', text)
    return text

# for text in texts[:5]:
#     print(keep_arabic_punctuations_numbers(text))

## [3.7] 

In [321]:
stop_words=set(stop_words.classed_stopwords_list())
Some_stop_words_we_need=["قبلما","بعدما","فقط","مازال","ليست","ليس","ليسا","لسنا",
                         "لسن","لازالت","مساء","صباح","قبل","بعد","إياك","إياكن",
                         "إياكما","إياكم","لم","عدا","إلا","د","كلا","عامة","لا",
                         "حبذا","أقل","أكثر"]
Some_stop_words_dont_we_need=['عليكم','السلام','انا','أرجو','الرد','المزيد','إقرأ']
for word in Some_stop_words_we_need:
    if word in stop_words:
        stop_words.remove(word)

for word in Some_stop_words_we_need:
    if word is not stop_words:
        stop_words.add(word)

def remove_stop_words(text):
    text =re.sub("[\_\-\/]"," ",text)
    text=word_tokenize(text)
    text=[w for w in text if not w in stop_words]
    return text

# for text in texts[:5]:
#     print(remove_stop_words(text))

ملاحــــــــــــــــــظه


## [3.8] 

In [322]:
ar_stemmer=stemmer("arabic")
ar_stemmer.stemWord(u"فسميتموها")

'سمي'

In [359]:
def stem_arabic_words_farasa(text):
    stemmer = FarasaStemmer()
    text=stemmer.stem(text)
    return text

text = "الكتب في المكتبة متنوعة ومفيدة"
stemmed_text = stem_arabic_words_farasa(text)
print(stemmed_text)

كتاب في مكتبة متنوع مفيد


In [324]:
# def stem_arabic_words_tashaphyne(text):
#     stemmer = ArabicLightStemmer()
#     words = text.split()
#     stemmed_words = [stemmer.light_stem(word) for word in words]
#     return ' '.join(stemmed_words)

# text = "الكتب في المكتبة متنوعة ومفيدة"
# stemmed_text = stem_arabic_words_tashaphyne(text)
# print(stemmed_text)

In [361]:
def stem_arabic_words_snowballstemmer(text):
    ar_stemmer = stemmer("arabic")
    words = text.split()
    stemmed_words = [ar_stemmer.stemWord(word) for word in words]
    return ' '.join(stemmed_words)

text = "الكتب في المكتبة متنوعة ومفيدة"
stemmed_text = stem_arabic_words_snowballstemmer(text)
print(stemmed_text)

كتب في مكتب متنوع مفيد


In [362]:
for text in texts[:5]:
    print(stem_arabic_words_snowballstemmer(text))

راجع طبيب من اجر جراحه افضل من يجيب لان شاهد حاله عل وضع اول وهو من قام لفعل تمنيات لشفاء 0 2023-06-17 05:56:49 دكتور حسن ابراهيم خليف /اسيلة-طبية/جراحة-عامة/عملت-عملية-دوالي-الساقين-قسطرة-الليزر-من-شهر-وعندي-1715059 دكتور حسن ابراهيم خليف جراح عام
لا يوحد مشكل مشي. اكثار من شرب سوايل ومم استخدام ملين والمغاطس دافء مع ملح 1 2018-03-13 08:07:00 دكتور اسام عبدالله /اسيلة-طبية/جراحة-عامة/عملت-عن-البواسير-شهر-وعانيت-خلال-التبرز-كثيرا-حدث-لي-1113406 دكتور اسام عبدالله جراح عام
اوافق علي را د/ محمدالشيخ و ايض ارجو مراجع طبيب باطن لضبط سكر بالدم 0 2022-08-13 10:24:38 د. سوزان حسن /اسيلة-طبية/الاورام-الخبيثة-والحميدة/حدث-معي-قبل-او-شيء-ك-تنميل-في-الوجه-وفي-اليد-وبعده-1610184 د. سوزان حسن اورام حميد و خبيث
بعد تمام بلوغ لايحدث نمو في طول 1 2015-08-29 20:56:25 دكتور احمد اسام عبدالغني صقر /اسيلة-طبية/جراحة-العظام/انا-بنت-و-لدي-مشكلة-هي-انني-قصيرة-القامة-من-الطول-سم-هل-525552 دكتور احمد اسام عبدالغني صقر جراح عظام والمفاصل
ليس خطير لكن يتوافق مع نقص في يتام ب 12 0 2016-04-12 08:18:38 دكتور اسماعي

In [327]:
# for text in texts[:5]:
#     print(stem_arabic_words_tashaphyne(text))

In [360]:
# for text in texts[:5]:
#     print(stem_arabic_words_farasa(text))

راجع طبيب من أجري جرح أفضل من أجاب لان شاهد حال علي وضع أول هو من قام فعل تمني بلشفاء
0
2023-06-17 05 : 56 : 49


دكتور حسن إبراهيم خليفة







/ سؤال - طبي / جراحة - عام / عمل - عملية - دالية - ساق - قسطرة - ليزر - من - شهر - عند - 1715059

دكتور حسن إبراهيم خليفة


جراحة عام
لا وحد مشكلة مشي . اكثار من شرب سائل ممكن استخدام ملين مغاطس دافئ مع ملح
1
2018-03-13 08 : 07 : 00


دكتور أسامة عبدالله







/ سؤال - طبي / جراحة - عام / عمل - عن - باسور - شهر - عانى - خلال - برز - كثير - حدث - ل - 1113406

دكتور أسامة عبدالله


جراحة عام
أوافق على راي د / محمدالشيخ و ايض أرجو مراجعة طبيب باطني ضبط سكر دم  
0
2022-08-13 10 : 24 : 38


د . سوزان حسنين







/ سؤال - طبي / اورام - خبيث - حميدة / حدث - مع - قبل - أو - شيء - ك - تنميل - في - وجه - في - يد - بعد - 1610184

د . سوزان حسنين


اورام حميدة و خبيث
بعد تمام بلوغ لايحدث نمو في طول
1
2015-08-29 20 : 56 : 25


دكتور أحمد أسامة عبدالغنى صقر







/ سؤال - طبي / جراحة - عظمة / أنا - بنت - و - لدى - مشكلة - هي - انني - قصير - قامة - من - ط

## [3.9] 

### [3.9.1] 

In [329]:
def unify_hamzat(text):
    unified_text = re.sub(r'[ءؤئ]', 'ء', text)
    return unified_text

In [330]:
text = "أحمد قرأ القرآن بتأنٍ وإتقان"
unified_text = unify_hamzat(text)
print(unified_text)

أحمد قرأ القرآن بتأنٍ وإتقان


### [3.9.2] 

In [331]:
def unify_Alfat(text):
    unified_text = re.sub(r'[أإآ]', 'أ', text)
    return unified_text

In [332]:
text = "أحمد قرأ القرآن بتأنٍ وإتقان"
unified_text = unify_Alfat(text)
print(unified_text)

أحمد قرأ القرأن بتأنٍ وأتقان


### [3.9.3] 

In [333]:
def remove_tatweel(text):
    text =re.sub( r'ـ', '', text)
    return text

In [334]:
text = "اللغة العربيــــــة جميلة"
clean_text = remove_tatweel(text)
print(clean_text)

اللغة العربية جميلة


### [3.9.4] 

In [335]:
def remove_arabic_diacritics(text):
    text=re.sub(r'[\u064B-\u065F]', '', text)
    return text


In [336]:
# for text in texts[:5]:
#     print(remove_arabic_diacritics(text))

## [3.10] 

In [337]:
def remove_space(text):
    text = re.sub(r"\s+", " ", text).strip()
    return text

## Extra [3.11]

# Question [4]: Prepare Training Data

## [4.1] 


In [338]:
def preparing_training_data(text):
    text_after_cleaning_links = remove_links(text)
    text_after_removing_doctors_prefix = remove_doctors_prefix(text_after_cleaning_links)
    text_after_removing_specialization = remove_sentences_after_doctor(text_after_removing_doctors_prefix)
    text_after_removing_doublication_chars = remove_duplicate_letters_except_alif(text_after_removing_specialization)
    text_after_removing_date_and_time = remove_dates_and_times(text_after_removing_doublication_chars)
    text_after_removing_numbers = remove_numbers(text_after_removing_date_and_time)
    text_after_removing_non_arabic_words = keep_arabic_punctuations_numbers(text_after_removing_numbers)
    # text after removing stop words
    text_after_unify_hamzat = unify_hamzat(text_after_removing_non_arabic_words)
    text_after_unify_alafat = unify_Alfat(text_after_unify_hamzat)
    text_after_removing_tatwel = remove_tatweel(text_after_unify_alafat)
    text_after_removing_tashkel = remove_arabic_diacritics(text_after_removing_tatwel)
    text_after_removing_spaces_and_keeping_one_space = remove_space(text_after_removing_tashkel)
    # text after removing wrong arabic words and correct them


    return text_after_removing_spaces_and_keeping_one_space
    
    
    
questions = train_data['question'].values
answers = train_data['answer'].values

train_data['cleann_questions']=questions
train_data['cleann_answer']=answers
questions_answers = questions[0:5] + answers[0:5]

questions_answers.tolist()
# for i in questions_answers:
#     print(preparing_training_data(i))
    
    

['\r\nما هي مميزات و عيوب الدواء جلوكوفانس 500 5 و ايضا الأنسولين مكس تارد 30 \r\n\r\nلكل علاج ايجابيته وسلبياته والتي تعتمد على حالة المريض فما قد يناسب احدهم قد لا يناسب الاخر وهذا يحدده الطبيب خلال مراجعة المريض له\r\n1\r\n2015-01-04 20:08:51\r\n\r\n\r\nالدكتور يزن علي خليف\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n/اسئلة-طبية/مرض-السكري/ما-هي-مميزات-و-عيوب-الدواء-جلوكوفانس-و-ايضا-الانسولين-351157\r\n\r\nالدكتور يزن علي خليف \r\n\r\n\r\nالغدد الصماء \r\n\r\n\r\n',
 '\r\nاليك نتيجة تحليل هرمونات الغدة الدرقية علما بانه تم استأصال الغدة منذ اكثر من سنتينTT3=163TT4=12.6TSH=.01 هل مطلوب تعديل الجرعة \r\n\r\nنعم. يجب تخفيض الجرعة، الا اذا كان سبب استئصال الغدة هو سرطان الغدة\r\n0\r\n2017-02-13 06:34:19\r\n\r\n\r\nالدكتور باسم مرقص\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n/اسئلة-طبية/امراض-الغدد-الصماء/اليك-نتيجة-تحليل-هرمونات-الغدة-الدرقية-علما-بانه-تم-909154\r\n\r\nالدكتور باسم مرقص \r\n\r\n\r\nجراحة عامة \r\n\r\n\r\n',
 '\r\nحلول منزلية لأعراض ارتفاع ضغط الدم \r\n\r\nيفضل عدم الاستغناء عن العلاج الدوائي لم

## [4.2] 

In [339]:
all_train_data = questions + answers
all_train_data_After_cleaning = []
all_tokens = []
for line in all_train_data:
    training_data_after_cleaning = preparing_training_data(line)
    all_train_data_After_cleaning.append(training_data_after_cleaning)
    # print(training_data_after_cleaning)


# all_train_data_After_cleaning
# # print(tokens)
for phrase in all_train_data_After_cleaning:
    tokens = nltk.word_tokenize(phrase)
    all_tokens.append(tokens)   




flattened_all_tokens = [item for sublist in all_tokens for item in sublist]
print(len(flattened_all_tokens)) 

1924068


In [340]:
# حساب عدد الكلمات الفريدة :
total_unique_words=len(set(flattened_all_tokens))
print("عدد الكلمات الفريدة:", total_unique_words)

عدد الكلمات الفريدة: 126989


In [341]:
# نلاحظ ان عدد الكلمات الكلية قد قلت هنا أما عدد الكلمات المميزة قد زاد

## [4.3] 

In [342]:

df = pd.DataFrame(all_train_data_After_cleaning)

df = df.map(lambda x: '' if isinstance(x, (float, int)) else x.strip() if isinstance(x, str) and len(x) >= 5 else None)

df = df.dropna()
print(len(df))
print(len(all_train_data_After_cleaning))
# all_train_data_After_cleaning


32598
32598


# Question [5]: Baseline

The Comparison Dictionary example

In [343]:
comparison_table = {}

comparison_table['question_step_number'] = []
comparison_table['model_name'] = []
comparison_table['features'] = []
comparison_table['model_parameters'] = []
comparison_table['preprocessing_methods'] = []
comparison_table['accuracy'] = []
comparison_table['balance_accuracy'] = []

Filling the dictionary example

In [344]:
# comparison_table['question_step_number'].append("3.5")
# comparison_table['model_name'].append('logistic_regression')
# comparison_table['features'].append('bag of words')
# comparison_table['model_parameters'].append('default')
# comparison_table['preprocessing_methods'].append("remove links")
# comparison_table['accuracy'].append(accuracy)
# comparison_table['balance_accuracy'].append(balance_accuracy_value)

## [5.1]

In [345]:
X_train, y_train = train_data[['question', 'answer']], train_data['label']
X_test, y_test = test_data[['question', 'answer']], test_data['label']
X_validation, y_validation = validation_data[['question', 'answer']], validation_data['label']


## [5.2]

In [346]:
X_train_combined = X_train['question'] + ' ' + X_train['answer']
X_test_combined = X_test['question'] + ' ' + X_test['answer']
X_validation_combined = X_validation['question'] + ' ' + X_validation['answer']


def logistic_regression(x_train,x_test,x_val):

    vectorizer = CountVectorizer()

    X_train_bow = vectorizer.fit_transform(x_train)
    X_test_bow = vectorizer.transform(x_test)
    X_validation_bow = vectorizer.transform(x_val)

    logreg_model = LogisticRegression(max_iter=1000)

    logreg_model.fit(X_train_bow, y_train)

    y_test_pred = logreg_model.predict(X_test_bow)
    y_validation_pred = logreg_model.predict(X_validation_bow)

    accuracy_test = accuracy_score(y_test, y_test_pred)
    accuracy_validation = accuracy_score(y_validation, y_validation_pred)
    balanced_accuracy_test = balanced_accuracy_score(y_test, y_test_pred)


    print(f"Test Accuracy: {accuracy_test}")
    print(f"Validation Accuracy: {accuracy_validation}")
    print(f"Balanced Accuracy: {balanced_accuracy_test}")


    return accuracy_test, accuracy_validation, balanced_accuracy_test


In [347]:
accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

comparison_table['question_step_number'].append("5.2")
comparison_table['model_name'].append('logistic_regression')
comparison_table['features'].append('bag of words')
comparison_table['model_parameters'].append('default')
comparison_table['preprocessing_methods'].append("nothing")
comparison_table['accuracy'].append(accuracy_test)
comparison_table['balance_accuracy'].append(balanced_accuracy_test)

Test Accuracy: 0.6437688353128878
Validation Accuracy: 0.6507362666431532
Balanced Accuracy: 0.6249945192325319



ملاحظااااااااااااااااااااااااااااااااااات




نلاحظ ان ال test accuracy دقتو أكثر من ال balanced accuracy بنسبة بسيطة


## [5.3]

In [348]:
# تنظيف النص من الروابط

X_train_combined = X_train['question'].apply(remove_links) + ' ' + X_train['answer'].apply(remove_links)


# print(len(X_train_without_links))
print(len(X_train_combined))

accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

comparison_table['question_step_number'].append("5.3")
comparison_table['model_name'].append('logistic_regression')
comparison_table['features'].append('bag of words')
comparison_table['model_parameters'].append('default')
comparison_table['preprocessing_methods'].append("Removing Links")
comparison_table['accuracy'].append(accuracy_test)
comparison_table['balance_accuracy'].append(balanced_accuracy_test)

 

32598
Test Accuracy: 0.6434142882467647
Validation Accuracy: 0.6510889692266996
Balanced Accuracy: 0.6247697063643611


In [349]:
# حذف اسماء الأطباء مع اللقب
X_train_combined = X_train['question'].apply(remove_doctors_prefix) + ' ' + X_train['answer'].apply(remove_doctors_prefix)


# print(len(X_train_without_links))
print(len(X_train_combined))

accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

comparison_table['question_step_number'].append("5.3")
comparison_table['model_name'].append('logistic_regression')
comparison_table['features'].append('bag of words')
comparison_table['model_parameters'].append('default')
comparison_table['preprocessing_methods'].append("ٌRemoving doctors prefix")
comparison_table['accuracy'].append(accuracy_test)
comparison_table['balance_accuracy'].append(balanced_accuracy_test)



32598
Test Accuracy: 0.639691544052473
Validation Accuracy: 0.6475619433912353
Balanced Accuracy: 0.6181974031602561


In [350]:
# حذف المحارف المكررة من النص
X_train_combined = X_train['question'].apply(remove_duplicate_letters_except_alif) + ' ' + X_train['answer'].apply(remove_duplicate_letters_except_alif)


# print(len(X_train_without_links))
print(len(X_train_combined))

accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

comparison_table['question_step_number'].append("5.3")
comparison_table['model_name'].append('logistic_regression')
comparison_table['features'].append('bag of words')
comparison_table['model_parameters'].append('default')
comparison_table['preprocessing_methods'].append("Removing duplicate letters except alif")
comparison_table['accuracy'].append(accuracy_test)
comparison_table['balance_accuracy'].append(balanced_accuracy_test)



32598
Test Accuracy: 0.6353483424924659
Validation Accuracy: 0.640243364782647
Balanced Accuracy: 0.6191241254584431


In [351]:
# تحويل الارقام الى العربية
X_train_combined = X_train['question'].apply(convert_numbers_to_arabic_v2) + ' ' + X_train['answer'].apply(convert_numbers_to_arabic_v2)


# print(len(X_train_without_links))
print(len(X_train_combined))

accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

comparison_table['question_step_number'].append("5.3")
comparison_table['model_name'].append('logistic_regression')
comparison_table['features'].append('bag of words')
comparison_table['model_parameters'].append('default')
comparison_table['preprocessing_methods'].append("Removing Links")
comparison_table['accuracy'].append(accuracy_test)
comparison_table['balance_accuracy'].append(balanced_accuracy_test)



32598
Test Accuracy: 0.6424392838149264
Validation Accuracy: 0.6497663345384005
Balanced Accuracy: 0.6240931361367352


In [352]:
# تحويل الارقام الى رمز من اختياري
X_train_combined = X_train['question'].apply(standardize_numbers) + ' ' + X_train['answer'].apply(standardize_numbers)


# print(len(X_train_without_links))
print(len(X_train_combined))

accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

comparison_table['question_step_number'].append("5.3")
comparison_table['model_name'].append('logistic_regression')
comparison_table['features'].append('bag of words')
comparison_table['model_parameters'].append('default')
comparison_table['preprocessing_methods'].append("standardize numbers to specific char")
comparison_table['accuracy'].append(accuracy_test)
comparison_table['balance_accuracy'].append(balanced_accuracy_test)



32598
Test Accuracy: 0.6348165218932813
Validation Accuracy: 0.6417423507627193
Balanced Accuracy: 0.6157853154416597


In [353]:
# حذف الأرقام كلها
X_train_combined = X_train['question'].apply(remove_numbers) + ' ' + X_train['answer'].apply(remove_numbers)


# print(len(X_train_without_links))
print(len(X_train_combined))

accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

comparison_table['question_step_number'].append("5.3")
comparison_table['model_name'].append('logistic_regression')
comparison_table['features'].append('bag of words')
comparison_table['model_parameters'].append('default')
comparison_table['preprocessing_methods'].append("Removing numbers")
comparison_table['accuracy'].append(accuracy_test)
comparison_table['balance_accuracy'].append(balanced_accuracy_test)



32598
Test Accuracy: 0.6348165218932813
Validation Accuracy: 0.6407724186579666
Balanced Accuracy: 0.6160329300852934


In [354]:
# حذف التاريخ والوقت
X_train_combined = X_train['question'].apply(remove_dates_and_times) + ' ' + X_train['answer'].apply(remove_dates_and_times)


# print(len(X_train_without_links))
print(len(X_train_combined))

accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

comparison_table['question_step_number'].append("5.3")
comparison_table['model_name'].append('logistic_regression')
comparison_table['features'].append('bag of words')
comparison_table['model_parameters'].append('default')
comparison_table['preprocessing_methods'].append("Removing Date and time")
comparison_table['accuracy'].append(accuracy_test)
comparison_table['balance_accuracy'].append(balanced_accuracy_test)



32598
Test Accuracy: 0.6217869172132601
Validation Accuracy: 0.628163301296182
Balanced Accuracy: 0.5997778691628503


In [355]:
# حذف المحارف الغير عربية
X_train_combined = X_train['question'].apply(keep_arabic_punctuations_numbers) + ' ' + X_train['answer'].apply(keep_arabic_punctuations_numbers)


# print(len(X_train_without_links))
print(len(X_train_combined))

accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

comparison_table['question_step_number'].append("5.3")
comparison_table['model_name'].append('logistic_regression')
comparison_table['features'].append('bag of words')
comparison_table['model_parameters'].append('default')
comparison_table['preprocessing_methods'].append("Remove non arabic words")
comparison_table['accuracy'].append(accuracy_test)
comparison_table['balance_accuracy'].append(balanced_accuracy_test)



32598
Test Accuracy: 0.6435915617798262
Validation Accuracy: 0.6483555242042148
Balanced Accuracy: 0.6242531728086279


In [356]:
# حذف كلمات التوقف
X_train_combined = X_train['question'].apply(keep_arabic_punctuations_numbers) + ' ' + X_train['answer'].apply(keep_arabic_punctuations_numbers)


# print(len(X_train_without_links))
print(len(X_train_combined))

accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

comparison_table['question_step_number'].append("5.3")
comparison_table['model_name'].append('logistic_regression')
comparison_table['features'].append('bag of words')
comparison_table['model_parameters'].append('default')
comparison_table['preprocessing_methods'].append("Remove non arabic words")
comparison_table['accuracy'].append(accuracy_test)
comparison_table['balance_accuracy'].append(balanced_accuracy_test)



32598
Test Accuracy: 0.6435915617798262
Validation Accuracy: 0.6483555242042148
Balanced Accuracy: 0.6242531728086279


In [363]:
# تجذير الكلمات العربية
X_train_combined = X_train['question'].apply(stem_arabic_words_snowballstemmer) + ' ' + X_train['answer'].apply(stem_arabic_words_snowballstemmer)


# print(len(X_train_without_links))
print(len(X_train_combined))

accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

comparison_table['question_step_number'].append("5.3")
comparison_table['model_name'].append('logistic_regression')
comparison_table['features'].append('bag of words')
comparison_table['model_parameters'].append('default')
comparison_table['preprocessing_methods'].append("stem arabic words")
comparison_table['accuracy'].append(accuracy_test)
comparison_table['balance_accuracy'].append(balanced_accuracy_test)



32598
Test Accuracy: 0.553270696684985
Validation Accuracy: 0.5588572436293096
Balanced Accuracy: 0.5175959620880198


In [364]:
# توحيد طريقة كتابة الهمزات
X_train_combined = X_train['question'].apply(unify_hamzat) + ' ' + X_train['answer'].apply(unify_hamzat)


# print(len(X_train_without_links))
print(len(X_train_combined))

accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

comparison_table['question_step_number'].append("5.3")
comparison_table['model_name'].append('logistic_regression')
comparison_table['features'].append('bag of words')
comparison_table['model_parameters'].append('default')
comparison_table['preprocessing_methods'].append("unify hamzat")
comparison_table['accuracy'].append(accuracy_test)
comparison_table['balance_accuracy'].append(balanced_accuracy_test)



32598


KeyboardInterrupt: 

In [ ]:
# توحيد طريقة كتابة الألفات
X_train_combined = X_train['question'].apply(unify_Alfat) + ' ' + X_train['answer'].apply(unify_Alfat)


# print(len(X_train_without_links))
print(len(X_train_combined))

accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

comparison_table['question_step_number'].append("5.3")
comparison_table['model_name'].append('logistic_regression')
comparison_table['features'].append('bag of words')
comparison_table['model_parameters'].append('default')
comparison_table['preprocessing_methods'].append("unify Alafat")
comparison_table['accuracy'].append(accuracy_test)
comparison_table['balance_accuracy'].append(balanced_accuracy_test)



In [ ]:
# حذف التطويل
X_train_combined = X_train['question'].apply(remove_tatweel) + ' ' + X_train['answer'].apply(remove_tatweel)


# print(len(X_train_without_links))
print(len(X_train_combined))

accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

comparison_table['question_step_number'].append("5.3")
comparison_table['model_name'].append('logistic_regression')
comparison_table['features'].append('bag of words')
comparison_table['model_parameters'].append('default')
comparison_table['preprocessing_methods'].append("remove tatweel")
comparison_table['accuracy'].append(accuracy_test)
comparison_table['balance_accuracy'].append(balanced_accuracy_test)



In [ ]:
# حذف التشكيل
X_train_combined = X_train['question'].apply(remove_arabic_diacritics) + ' ' + X_train['answer'].apply(remove_arabic_diacritics)


# print(len(X_train_without_links))
print(len(X_train_combined))

accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

comparison_table['question_step_number'].append("5.3")
comparison_table['model_name'].append('logistic_regression')
comparison_table['features'].append('bag of words')
comparison_table['model_parameters'].append('default')
comparison_table['preprocessing_methods'].append("remove arabic diacritics")
comparison_table['accuracy'].append(accuracy_test)
comparison_table['balance_accuracy'].append(balanced_accuracy_test)



In [ ]:
# حذف الفراغات
X_train_combined = X_train['question'].apply(remove_space) + ' ' + X_train['answer'].apply(remove_space)


# print(len(X_train_without_links))
print(len(X_train_combined))

accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

comparison_table['question_step_number'].append("5.3")
comparison_table['model_name'].append('logistic_regression')
comparison_table['features'].append('bag of words')
comparison_table['model_parameters'].append('default')
comparison_table['preprocessing_methods'].append("remove spaces with one space")
comparison_table['accuracy'].append(accuracy_test)
comparison_table['balance_accuracy'].append(balanced_accuracy_test)



In [ ]:
# حذف اختصاص الطبيب
X_train_combined = X_train['question'].apply(remove_sentences_after_doctor) + ' ' + X_train['answer'].apply(remove_sentences_after_doctor)


# print(len(X_train_without_links))
print(len(X_train_combined))

accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(X_train_combined,X_test_combined,X_validation_combined)

comparison_table['question_step_number'].append("5.3")
comparison_table['model_name'].append('logistic_regression')
comparison_table['features'].append('bag of words')
comparison_table['model_parameters'].append('default')
comparison_table['preprocessing_methods'].append("remove doctore specialization")
comparison_table['accuracy'].append(accuracy_test)
comparison_table['balance_accuracy'].append(balanced_accuracy_test)



## [5.4]

## [5.5]

In [ ]:
tfidf_vectorizer  = TfidfVectorizer(min_df=2,max_df=0.5,ngram_range=(1,2))
train_data_X_train_combined = np.array(X_train_combined)
train_data_X_test_combined = np.array(X_test_combined)
train_data_X_validation_combined = np.array(X_validation_combined)
tfidf_matrix_X_train_combined = tfidf_vectorizer .fit_transform(train_data_X_train_combined)
tfidf_matrix_X_test_combined = tfidf_vectorizer .fit_transform(train_data_X_test_combined)
tfidf_matrix_X_validation_combined = tfidf_vectorizer .fit_transform(train_data_X_validation_combined)

In [ ]:
accuracy_test, accuracy_validation, balanced_accuracy_test = logistic_regression(tfidf_matrix_X_train_combined,tfidf_matrix_X_test_combined,tfidf_matrix_X_validation_combined)


In [ ]:
X_test = tfidf_vectorizer.transform(X_test)
predicted_labels = model.predict(X_test)

## [5.6]

# Question [6]: Semantic Representation

## [6.1]

## [6.2]

## [6.3]

## [6.4]

## [6.5]

# Question [7]: Deep Neural Network

## [7.1]

### [7.1.1]

### [7.1.2]

## [7.2]

### [7.2.1]

### [7.2.2]

### [7.2.3]

### [7.2.4]

## [7.3]

## Extra [7.4]

# Final Results

In [ ]:
df = pd.DataFrame(comparison_table)
df

In [ ]:
df.to_csv("[your_name].csv", index=False)